In [ ]:
!pip install faiss-cpu tqdm openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 18.3 MB/s eta 0:00:00


In [ ]:
!pip install -U langchain-community faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.11
    Uninstalling langchain-text-splitters-0.3.11:
      Successfully uninstalled langchain-text-splitters-0.3.11
ERROR: pip's dependency resolver

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from openai import OpenAI
from langchain_community.vectorstores import FAISS
from langchain.embeddings.base import Embeddings
from tqdm import tqdm
import json, time, os

# ======================================================
# 1) Upstage Client 연결
# ======================================================
client = OpenAI(
    api_key="up_m5U58N6ugmo67EJWWRnoKUcCvYFqK",
    base_url="https://api.upstage.ai/v1"
)

# ======================================================
# 2) LangChain 호환 Embedding Wrapper
# ======================================================
class SolarEmbedding(Embeddings):

    def embed_query(self, text: str):
        res = client.embeddings.create(
            model="solar-embedding-1-large-query",   # 쿼리용
            input=text
        )
        return res.data[0].embedding

    def embed_documents(self, texts, batch_size=8):
        all_embeddings = []

        for i in tqdm(range(0, len(texts), batch_size)):
            batch = texts[i:i + batch_size]

            res = client.embeddings.create(
                model="solar-embedding-1-large-passage",
                input=batch
            )
            batch_embeddings = [d.embedding for d in res.data]
            all_embeddings.extend(batch_embeddings)

        return all_embeddings

# ======================================================
# 3) JSON → Embedding → FAISS
# --------------------------
def embed_json(json_path, save_dir="ewha_vectorstore"):
    # jsonl 또는 json 자동 처리
    chunks = []
    with open(json_path, "r", encoding="utf-8") as f:
        first = f.readline().strip()
        f.seek(0)
        if first.startswith("{"):
            try:
                chunks = json.load(f)
            except:
                f.seek(0)
                chunks = [json.loads(line)["text"] for line in f]
        else:
            raise ValueError("File format invalid.")

    print(f"✅ Loaded {len(chunks)} chunks")

    emb_model = SolarEmbedding()

    # batch embedding
    embeddings = emb_model.embed_documents(chunks)
    print("✅ Embedding complete!")

    # FAISS 저장
    vectorstore = FAISS.from_texts(chunks, emb_model)
    vectorstore.save_local(save_dir)
    print(f"🎉 Saved FAISS vectorstore to {save_dir}/")


# --------------------------
# 4) Run
# --------------------------
json_path = "/content/drive/MyDrive/NLP_TermProject/chunking/ewha_chunks_by_article.jsonl"
embed_json(json_path)

✅ Loaded 242 chunks


100%|██████████| 31/31 [00:41<00:00,  1.35s/it]


✅ Embedding complete!


100%|██████████| 31/31 [00:43<00:00,  1.41s/it]


🎉 Saved FAISS vectorstore to ewha_vectorstore/


In [ ]:
from openai import OpenAI
from langchain_community.vectorstores import FAISS
from langchain.embeddings.base import Embeddings
from tqdm import tqdm
import json, time, os

# Upstage Client 생성
client = OpenAI(
    api_key="up_m5U58N6ugmo67EJWWRnoKUcCvYFqK",
    base_url="https://api.upstage.ai/v1"
)

# LangChain 호환 Embedding Wrapper
class SolarEmbedding(Embeddings):

    def embed_query(self, text: str):
        res = client.embeddings.create(
            model="solar-embedding-1-large-query",
            input=text
        )
        return res.data[0].embedding

    def embed_documents(self, texts, batch_size=8):
        all_embeddings = []
        for i in tqdm(range(0, len(texts), batch_size)):
            batch = texts[i:i+batch_size]
            res = client.embeddings.create(
                model="solar-embedding-1-large-passage",
                input=batch
            )
            all_embeddings.extend([d.embedding for d in res.data])
        return all_embeddings


In [ ]:
# 저장된 벡터스토어 경로 (전체 구조 기반)
vectorstore_path = "/content/ewha_vectorstore"

# FAISS 인덱스 로드
embedding_model = SolarEmbedding()
db = FAISS.load_local(
    vectorstore_path,
    embedding_model,
    allow_dangerous_deserialization=True
)

retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

print("✅ Retriever ready!")

✅ Retriever ready!


In [ ]:
def ask_solar(question, retrieved_docs):
    """
    question: 사용자의 질문
    retrieved_docs: top-k 문단 문자열 리스트
    """
    context = "\n\n".join([f"- {doc}" for doc in retrieved_docs])

    prompt = f"""
사용자 질문에 답해주세요. 문맥은 아래와 같습니다.
답변은 반드시 최종에 "[ANSWER]: (A) ..." 형태로 파싱 가능하게 해주세요.

[컨텍스트]
{context}

[질문]
{question}
    """

    response = client.chat.completions.create(
        model="solar-pro2",
        messages=[{"role":"user","content":prompt}],
        # instruction에 따라 reasoning=False (chat-only)
    )

    return response.choices[0].message.content

In [ ]:
def rag_answer(query):
    # retriever로 관련 문서 검색
    docs = retriever.invoke(query)

    # 문서 내용을 하나의 context로 결합
    context = "\n\n".join([d.page_content for d in docs])

    # 모델 호출
    completion = client.chat.completions.create(
        model="solar-pro",
        messages=[
            {"role": "system", "content": "You must answer ONLY based on the context."},
            {"role": "user", "content": f"Context: {context}\n\nQuestion: {query}"}
        ]
    )

    return completion.choices[0].message.content

In [ ]:
def rag_answer(question):
    # 1) top-k 문단 검색
    docs = retriever.invoke(question)
    retrieved_texts = [d.page_content for d in docs]

    print("🔍 Retrieved docs:")
    for i, t in enumerate(retrieved_texts):
        print(f"\n--- Document #{i+1} ---\n{t[:300]}...\n")

    # 2) Solar LLM 호출
    answer = ask_solar(question, retrieved_texts)
    return answer

In [ ]:
question = "영어 및 정보 자격 취득 시 인정되는 학점은 몇 점인가?"

answer = rag_answer(question)
print("\n🟦 최종 RAG 답변:")
print(answer)

🔍 Retrieved docs:

--- Document #1 ---
제48조의2(영어 및 정보인증) ① 영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을
취득한 경우 이를 각 3학점으로 인정하고 인증서를 교부할 수 있다. (개정 2015.9.18.)
② 제1항의 시행에 관한 사항은 총장이 따로 정한다.
[본조신설 2000.6.20.] [제목개정 2015.9.18.] 2 - 2 - 12

이화여자대학교 학칙...


--- Document #2 ---
제48조(졸업에 필요한 학점) ① 졸업에 필요한 학점은 129학점으로 한다. (개정 2012.2.
29.)
② 제1항의 규정에 불구하고 총장은 교과과정상의 필요에 따라 특정 학과 또는 전공의 졸
업에 필요한 학점을 129학점 이상으로 정할 수 있다. (개정 2012.2.29)
③ 학생은 재학기간중 제1항 또는 제2항의 학점외에 훈련학점을 취득하여야 한다.
④ 삭제 (2000.6.20.)
⑤ 학생은 재학기간 중 제1항 또는 제2항의 학점 내에서 총장이 정하는 소정의 학점을 영
어강의로 이수하여야 한다. (신설 2013.11.20)
⑥...


--- Document #3 ---
제35조의3(학점의 인정) ① 다음 각 호의 경우에 취득한 학점은 총장의 승인을 얻어 졸업에
필요한 학점의 2분의 1의 범위 안에서 이를 본교에서 취득한 것으로 본다. (개정
2012.12.31.)
1. 재학 중 국내외의 다른 학교에서 학점을 취득한 경우
2. 입학 전 국내외의 고등학교와 「고등교육법」...


🟦 최종 RAG 답변:
[ANSWER]: (A) 각 3학점  

**해설**:  
제48조의2(영어 및 정보인증) ①항에 따르면, 영어 및 정보 분야에서 일정 기준 이상의 능력 또는 자격을 취득한 경우, **각 3학점**으로 인정하고 인증서를 교부할 수 있도록 규정되어 있습니다. 따라서 영어 자격과 정보 자격 각각 3학점씩 총 6학점이 인정될 수 있으나, 질문에서는 "영어 및 정보 자격 취득 시 인정되는 학점"을 단일 항목으로